In [1]:
import boto3
import zipfile
import os

# Especificar o perfil do AWS CLI
session = boto3.Session(profile_name='CRISTIAN_AWS')

# Criar cliente do Lambda com a sessão específica
lambda_client = session.client('lambda')

# Função para criar um arquivo zip contendo o código da Lambda
def criar_arquivo_zip():
    # Código da função Lambda
    codigo_lambda = """
import json
import boto3
import uuid
from datetime import datetime

# Conectar ao DynamoDB
dynamodb = boto3.resource('dynamodb')
tabela = dynamodb.Table('ContatoFormulario')

def lambda_handler(event, context):
    # Obter os dados do corpo da requisição (via API Gateway)
    body = json.loads(event['body'])

    nome = body.get('nome')
    email = body.get('email')
    assunto = body.get('assunto')
    mensagem = body.get('mensagem')

    # Gerar um ID único para o contato
    contato_id = str(uuid.uuid4())
    
    # Registrar a data de envio
    timestamp = datetime.utcnow().isoformat()

    # Dados a serem inseridos no DynamoDB
    item = {
        'ContatoID': contato_id,
        'Nome': nome,
        'Email': email,
        'Assunto': assunto,
        'Mensagem': mensagem,
        'DataEnvio': timestamp
    }

    # Inserir no DynamoDB
    try:
        tabela.put_item(Item=item)
        return {
            'statusCode': 200,
            'body': json.dumps({'message': 'Contato enviado com sucesso!'})
        }
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps({'error': str(e)})
        }
    """

    # Criar um arquivo ZIP contendo o código da Lambda
    with open("lambda_function.py", "w") as arquivo_lambda:
        arquivo_lambda.write(codigo_lambda)
    
    # Compactar o arquivo
    with zipfile.ZipFile("lambda_function.zip", "w") as zipf:
        zipf.write("lambda_function.py")

# Função para criar a Lambda na AWS
def criar_lambda():
    criar_arquivo_zip()
    
    # Ler o arquivo ZIP criado
    with open("lambda_function.zip", 'rb') as f:
        codigo_zip = f.read()

    try:
        response = lambda_client.create_function(
            FunctionName='ContatoFormularioFunction',
            Runtime='python3.9',
            Role='arn:aws:iam::058264285974:role/LambdaDynamoDBRole',  # Substitua pelo ARN correto da Role
            Handler='lambda_function.lambda_handler',
            Code={'ZipFile': codigo_zip},
            Description='Função Lambda para inserir dados do formulário de contato no DynamoDB',
            Timeout=15,
            MemorySize=128,
            Publish=True
        )
        print("Função Lambda criada com sucesso!")
        print(response)
    except Exception as e:
        print(f"Erro ao criar a função Lambda: {e}")

# Chamar a função para criar a Lambda
criar_lambda()


Função Lambda criada com sucesso!
{'ResponseMetadata': {'RequestId': '25eff6b4-f5fa-40ff-b36b-9607e9ee6f5c', 'HTTPStatusCode': 201, 'HTTPHeaders': {'date': 'Fri, 11 Oct 2024 03:44:05 GMT', 'content-type': 'application/json', 'content-length': '1431', 'connection': 'keep-alive', 'x-amzn-requestid': '25eff6b4-f5fa-40ff-b36b-9607e9ee6f5c'}, 'RetryAttempts': 0}, 'FunctionName': 'ContatoFormularioFunction', 'FunctionArn': 'arn:aws:lambda:us-east-1:058264285974:function:ContatoFormularioFunction', 'Runtime': 'python3.9', 'Role': 'arn:aws:iam::058264285974:role/LambdaDynamoDBRole', 'Handler': 'lambda_function.lambda_handler', 'CodeSize': 1353, 'Description': 'Função Lambda para inserir dados do formulário de contato no DynamoDB', 'Timeout': 15, 'MemorySize': 128, 'LastModified': '2024-10-11T03:44:05.366+0000', 'CodeSha256': '8KCR/ZnV9k4Sv5cfa5oHPbhnGfEvqxwdH4IbtR74Pl8=', 'Version': '1', 'TracingConfig': {'Mode': 'PassThrough'}, 'RevisionId': '250b80fb-f03e-4448-bb7f-abd9bd99ac05', 'State': 'P